Dataset source: https://www.kaggle.com/datasets/arianazmoudeh/airbnbopendata

Objectives:
1. The cleaned data will be used for analysis of airbnb houses with "no-smoking" policy in New York.
2. Get only columns that are relevant for data analysis. Remove columns: id, host name, lat, long, country, country code and license (since the id for each reviewer, host name and license are irrelevant for the analysis, and it only focuses on the local scene (neighbourhood area))
3. Remove duplicates (if there are any)
4. Check neighbourhood group, update inconsistencies in spelling (Brooklyn and Manhattan)
5. Fill null values with: 
   - "Unknown" (for categorical/object type columns: NAME, host_identity_verified, instant_bookable, cancellation_policy,  house_rules)
6. Under house_rules, only get data that has no smoking policy
7. Do not include rows with no price AND service fee (if there are any)
8. Add new columns: 
   "> 0 min nights", "<= 2022 last review"
   - Values: Yes/No
   - "> 0 min nights" column description: 
       - Yes for data in "minimum nights" column that are > 0, 
       - No if < 0
   - "<= 2022 last review" column description: 
       - Yes for data in "last review" column that are <= 2022, 
       - No if > 2022
   - Reason for adding: There are outlier values for the following numeric type columns. The newly added columns may be used in analysis by dividing them from other data.

In [1]:
import pandas as pd

In [2]:
# Read Airbnb csv file
df = pd.read_csv("Airbnb_Open_Data.csv")
# Check first 5 rows of Airbnb_Open_Data
df.head()

C:\Users\Emelgar Rey Jahsua\AppData\Local\Temp\ipykernel_4108\3731740676.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Airbnb_Open_Data.csv")


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [3]:
# Get only columns that are relevant for data analysis. 
# Remove columns: id, host name, lat, long, country, country code and license
unwanted_columns = ['id', 'host name', 'lat', 'long', 'country', 'country code', 'license']
df.drop(unwanted_columns, axis=1, inplace=True)
# Display updated dataset
display(df)

,NAME,host id,host_identity_verified,neighbourhood group,neighbourhood,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules
0,Clean & quiet apt home by the park,80014485718,unconfirmed,Brooklyn,Kensington,False,strict,Private room,2020.0,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...
1,Skylit Midtown Castle,52335172823,verified,Manhattan,Midtown,False,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...
2,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Manhattan,Harlem,True,flexible,Private room,2005.0,$620,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and..."
3,NaN,85098326012,unconfirmed,Brooklyn,Clinton Hill,True,moderate,Entire home/apt,2005.0,$368,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN
4,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Manhattan,East Harlem,False,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102053,Cozy bright room near Prospect Park,77326652202,unconfirmed,Brooklyn,Flatbush,True,moderate,Private room,NaN,$696,NaN,7.0,12.0,3/27/2019,0.44,5.0,1.0,0.0,Shoes off Clean After yourself Turn Lights and...
102054,Private Bedroom with Amazing Rooftop View,45936254757,verified,Brooklyn,Bushwick,False,flexible,Private room,NaN,$909,NaN,1.0,19.0,8/31/2017,0.72,3.0,2.0,0.0,#NAME?
102055,Pretty Brooklyn One-Bedroom for 2 to 4 people,23801060917,verified,Brooklyn,Bedford-Stuyvesant,True,moderate,Entire home/apt,NaN,$387,NaN,2.0,50.0,6/26/2019,3.12,4.0,2.0,235.0,* Check out: 10am * We made an effort to keep ...
102056,Room & private bathroom in historic Harlem,15593031571,unconfirmed,Manhattan,Harlem,True,strict,Private room,NaN,$848,NaN,2.0,0.0,NaN,NaN,1.0,1.0,0.0,Each of us is working and/or going to school a...


In [4]:
# Remove duplicates (if there are any)
# Check if there are any duplicates by grouping the data by column and getting each size
duplicated_rows = df.groupby(df.columns.tolist(), as_index=False).size()
display(duplicated_rows[duplicated_rows['size'] > 1])

,NAME,host id,host_identity_verified,neighbourhood group,neighbourhood,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,size


In [5]:
# There are no duplicates to remove
df.drop_duplicates(inplace=True)
print(df.shape[0])

102058


In [8]:
# Check neighbourhood group, update inconsistencies in spelling (Brooklyn and Manhattan)
# There are cases that Brooklyn is brookln and Manhattan is manhatan
print(df['neighbourhood group'].value_counts())

Manhattan        43558
Brooklyn         41631
Queens           13197
Bronx             2694
Staten Island      949
Name: neighbourhood group, dtype: int64


In [9]:
# Set to correct spelling
df[df['neighbourhood group'] == 'brookln'] = "Brooklyn"
df[df['neighbourhood group'] == 'manhatan'] = "Manhattan"
print(df['neighbourhood group'].value_counts())

Manhattan        43558
Brooklyn         41631
Queens           13197
Bronx             2694
Staten Island      949
Name: neighbourhood group, dtype: int64


In [10]:
# Fill null values with:
# "Unknown" (for categorical/object type columns: NAME, host_identity_verified, 
#            instant_bookable, cancellation_policy, house_rules)
categorical_columns = {'NAME': 'Unknown', 
                       'host_identity_verified': 'Unknown', 
                       'instant_bookable': 'Unknown', 
                       'cancellation_policy': 'Unknown', 
                       'house_rules': 'Unknown'}
df = df.fillna(categorical_columns)

In [11]:
# Check dataframe to check if there are no null values
print(df[['NAME', 'host_identity_verified', 'instant_bookable', 'cancellation_policy', 'house_rules']].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102058 entries, 0 to 102057
Data columns (total 5 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   NAME                    102058 non-null  object
 1   host_identity_verified  102058 non-null  object
 2   instant_bookable        102058 non-null  object
 3   cancellation_policy     102058 non-null  object
 4   house_rules             102058 non-null  object
dtypes: object(5)
memory usage: 4.7+ MB
None


In [12]:
# Under house_rules column, only get data that has no smoking policy
# Check values that contains: no, non, smoke, smoking, '-' in between words
smoke_kw_airbnb = df[df['house_rules'].str.contains('smok', case=False)]
no_smok_kw_airbnb = smoke_kw_airbnb[smoke_kw_airbnb['house_rules'].str.contains('no smok', case=False)]
no_dash_smok_kw_airbnb = smoke_kw_airbnb[smoke_kw_airbnb['house_rules'].str.contains('no-smok', case=False)]
non_smok_kw_airbnb = smoke_kw_airbnb[smoke_kw_airbnb['house_rules'].str.contains('non smok', case=False)]
non_dash_smok_kw_airbnb = smoke_kw_airbnb[smoke_kw_airbnb['house_rules'].str.contains('non-smok', case=False)]

# Merge dataframes and remove duplicates
merged_kw = pd.concat([no_smok_kw_airbnb, no_dash_smok_kw_airbnb, non_smok_kw_airbnb, non_dash_smok_kw_airbnb])
no_smoking_policy_airbnbs = merged_kw.drop_duplicates()

In [13]:
# Show no_smoking_policy_airbnbs table
display(no_smoking_policy_airbnbs)

,NAME,host id,host_identity_verified,neighbourhood group,neighbourhood,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules
0,Clean & quiet apt home by the park,80014485718,unconfirmed,Brooklyn,Kensington,False,strict,Private room,2020.0,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...
2,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,Unknown,Manhattan,Harlem,True,flexible,Private room,2005.0,$620,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and..."
4,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Manhattan,East Harlem,False,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,11/19/2018,0.1,3.0,1.0,289.0,"Please no smoking in the house, porch or on th..."
5,Large Cozy 1 BR Apartment In Midtown East,45498551794,verified,Manhattan,Murray Hill,True,flexible,Entire home/apt,2013.0,$577,$115,3.0,74.0,6/22/2019,0.59,3.0,1.0,374.0,"No smoking, please, and no drugs."
6,BlissArtsSpace!,61300605564,Unknown,Brooklyn,Bedford-Stuyvesant,False,moderate,Private room,2015.0,$71,$14,45.0,49.0,10/5/2017,0.4,5.0,1.0,224.0,Please no shoes in the house so bring slippers...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101759,First Floor Studio on the Upper East Side!,33411205113,unconfirmed,Manhattan,Upper East Side,True,moderate,Entire home/apt,2006.0,$809,$162,3.0,30.0,6/28/2019,1.08,1.0,1.0,4.0,"We live on the premises - two women, two dogs..."
101787,Spacious Bed and Bathroom in Brooklyn,82554454538,verified,Brooklyn,Sheepshead Bay,True,flexible,Private room,2017.0,$406,$81,1.0,1.0,2/25/2017,0.03,2.0,1.0,0.0,THIS IS A NON-SMOKING RESIDENCE. NO Children u...
101794,"Private Sunny Room in Williamsburg, Brooklyn - NY",12776982396,verified,Brooklyn,Williamsburg,True,moderate,Private room,2005.0,$736,$147,2.0,4.0,1/2/2019,0.14,5.0,2.0,0.0,"This is a non-smoking, pet friendly house. Ple..."
101855,Brooklyn Cave in the Neighb,93010217247,verified,Brooklyn,Canarsie,True,strict,Entire home/apt,2011.0,"$1,033",$207,2.0,68.0,7/1/2019,3.5,2.0,1.0,240.0,THIS IS A NON-SMOKING RESIDENCE. NO Children u...


In [14]:
# Do not include rows with no price AND service fee (if there are any)
# Count rows that have no price and service fees in no_smoking_policy_airbnbs
no_price_and_fees = no_smoking_policy_airbnbs[no_smoking_policy_airbnbs['price'].isnull() & 
                                                    no_smoking_policy_airbnbs['service fee'].isnull()]
print(f"There are {no_price_and_fees.shape[0]} Airbnb hosts with no smoking policy that has both no price and service fees.")

There are 7 Airbnb hosts with no smoking policy that has both no price and service fees.


In [26]:
# Remove no_price_and_fees rows
cleaned_no_smoking_airbnbs = no_smoking_policy_airbnbs.drop(no_price_and_fees.index)
print(cleaned_no_smoking_airbnbs.shape[0])

26340


In [27]:
# Reset Index
cleaned_no_smoking_airbnbs.reset_index(drop=True, inplace=True)

In [28]:
# Add new columns: "> 0 min nights", "<= 2022 last review"
# First, count rows that have < 0 min nights (outlier values) in the 'minimum nights' column
# The data used is not null
min_nights_notnull = cleaned_no_smoking_airbnbs[cleaned_no_smoking_airbnbs['minimum nights'].notnull()]
min_nights_outliers = min_nights_notnull[min_nights_notnull['minimum nights'] < 0]['minimum nights']
print(f" There are {min_nights_outliers.count()} rows with < 0 min nights")

 There are 4 rows with < 0 min nights


In [29]:
# Add column: "> 0 min nights"
# Yes for data in "minimum nights" column that are > 0, No if < 0
cleaned_no_smoking_airbnbs['> 0 min nights'] = min_nights_notnull['minimum nights'].apply(lambda x: 'Yes' if x > 0 else 'No')

In [30]:
# Check outliers if '> 0 min nights' column is 'No' 
display(cleaned_no_smoking_airbnbs[cleaned_no_smoking_airbnbs['minimum nights'] < 0])

,NAME,host id,host_identity_verified,neighbourhood group,neighbourhood,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,> 0 min nights
81,BROOKLYN VICTORIAN STYLE SUITE.....,83631499592,unconfirmed,Brooklyn,Fort Greene,False,flexible,Private room,2008.0,"$1,155",$231,-10.0,213.0,6/24/2019,2.0,5.0,2.0,19.0,There is no smoking on the property. No outsid...,No
14293,2bd BOUTIQUE Apartament in the heart of MANHA...,2679070022,unconfirmed,Manhattan,Hell's Kitchen,True,flexible,Entire home/apt,2009.0,$711,$142,-365.0,13.0,7/7/2019,5.91,4.0,4.0,0.0,Please pick up/drop packet with front desk con...,No
14300,Newly Renovated Garden Apartment,98469733112,verified,Brooklyn,Bedford-Stuyvesant,True,moderate,Entire home/apt,2022.0,$85,$17,-200.0,3.0,4/23/2019,1.06,2.0,1.0,157.0,No smoking or pets allowed and we request that...,No
21676,Cozy Brooklyn Apartment,63988893317,verified,Brooklyn,Gowanus,True,moderate,Shared room,2014.0,"$1,177",$235,-10.0,9.0,5/8/2017,0.31,2.0,1.0,0.0,"No smoking, No pets. No shoes in the house. Vi...",No


In [31]:
# Check last reviews whose year is > 2022 (outlier values) in the 'last review' column
# The data used is not null
last_review_not_null = cleaned_no_smoking_airbnbs[cleaned_no_smoking_airbnbs['last review'].notnull()]

# Split the date by / and get the 3rd index ([2])
last_review_year = last_review_not_null['last review'].str.split('/').str[2].astype(int)
# Display rows whose year is > 2022
display(last_review_not_null[last_review_year > 2022])

,NAME,host id,host_identity_verified,neighbourhood group,neighbourhood,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,> 0 min nights
61,Garden studio in the Upper East Sid,77172555024,unconfirmed,Manhattan,Upper East Side,False,strict,Entire home/apt,2007.0,$571,$114,5.0,21.0,8/15/2024,0.19,4.0,NaN,395.0,"Dear Guests, Welcome to 62 Cornwall St! I hope...",Yes
107,Beautiful Landmarked Duplex,87944779917,Unknown,Brooklyn,Greenpoint,True,moderate,Entire home/apt,2008.0,$842,$168,3.0,124.0,6/16/2058,1.22,4.0,3.0,230.0,No rules per say. I just ask that you respect ...,Yes
132,Unknown,70084472212,verified,Brooklyn,Greenpoint,True,flexible,Private room,2018.0,$920,$184,2.0,1.0,3/28/2026,0.01,3.0,1.0,73.0,Check-In is 3pm. Check-Out is 12 Noon. In or...,Yes


In [32]:
# Add column: "<= 2022 last review"
# Yes for data in "last review" column that are <= 2022, No if > 2022
cleaned_no_smoking_airbnbs['<= 2022 last review'] = last_review_year.apply(lambda x: 'Yes' if x <= 2022 else 'No')

In [33]:
# Check outliers if '<= 2022 last review' column is 'No' 
display(cleaned_no_smoking_airbnbs[cleaned_no_smoking_airbnbs['<= 2022 last review'] == 'No'])

,NAME,host id,host_identity_verified,neighbourhood group,neighbourhood,instant_bookable,cancellation_policy,room type,Construction year,price,...,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,> 0 min nights,<= 2022 last review
61,Garden studio in the Upper East Sid,77172555024,unconfirmed,Manhattan,Upper East Side,False,strict,Entire home/apt,2007.0,$571,...,5.0,21.0,8/15/2024,0.19,4.0,NaN,395.0,"Dear Guests, Welcome to 62 Cornwall St! I hope...",Yes,No
107,Beautiful Landmarked Duplex,87944779917,Unknown,Brooklyn,Greenpoint,True,moderate,Entire home/apt,2008.0,$842,...,3.0,124.0,6/16/2058,1.22,4.0,3.0,230.0,No rules per say. I just ask that you respect ...,Yes,No
132,Unknown,70084472212,verified,Brooklyn,Greenpoint,True,flexible,Private room,2018.0,$920,...,2.0,1.0,3/28/2026,0.01,3.0,1.0,73.0,Check-In is 3pm. Check-Out is 12 Noon. In or...,Yes,No


In [477]:
# Export cleaned_no_smoking_airbnbs as csv
cleaned_no_smoking_airbnbs.to_csv('cleaned_no_smoking_airbnbs.csv', index=False)